# 🧾 In-Memory Video Frame to PDF Generator

This script processes a video file to extract frames at regular intervals and compiles them into a single PDF document — **entirely in memory** (no intermediate image files saved to disk).

---

## 🧩 Parameters:
- **video_path** (`str`, required): Path to the video file.
- **save_path** (`str`, optional): Directory where the final PDF will be saved. Defaults to the current directory `"."`.
- **interval** (`int` or `float`, optional): Time in seconds between captured frames. Defaults to `1` second.

---

## ✅ What This Version Does:
- Captures video frames directly (no saving to disk).
- Converts each captured frame to a `PIL.Image` in memory.
- Stores all images in a list.
- Saves all images as a single multi-page PDF.

---

## ⚡ Performance Benefit:
- **Roughly 2× faster** than a version that saves each frame to disk (since we skip file I/O).
- Ideal for machines with slower disks (e.g., HDDs) or for faster batch operations.

---

## 🧠 Estimated Memory Usage (for 1920×1080 Full HD):
Each frame is:
- 1920 × 1080 × 3 bytes ≈ **6 MB per frame**

| Interval | Frames (2hr video) | Est. RAM Usage |
|----------|--------------------|----------------|
| 1 sec    | 7200               | ~43 GB ❌       |
| 5 sec    | 1440               | ~8.6 GB ✅       |
| 10 sec   | 720                | ~4.3 GB ✅       |

> ⚠️ For longer videos, prefer larger intervals (e.g., 5–10 seconds) unless you have lots of RAM.

---

## 📦 Requirements:
Install the necessary libraries with:

```bash
> pip install opencv-python pillow

In [1]:
import cv2
from PIL import Image
from typing import Dict
import os
import time


def video_to_pdf_in_memory(params: Dict[str, any]):
    video_path = params.get("video_path")
    save_path = params.get("save_path", ".")
    interval = params.get("interval", 1)

    os.makedirs(save_path, exist_ok=True)

    # Extract and sanitize filename (remove extension, replace spaces)
    video_filename = os.path.basename(video_path)
    base_name = os.path.splitext(video_filename)[0].replace(" ", "_")
    pdf_filename = base_name + ".pdf"
    pdf_path = os.path.join(save_path, pdf_filename)

    start_time = time.time()

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError(f"Could not open video file: {video_path}")

    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_interval = int(fps * interval)

    pil_images = []
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % frame_interval == 0:
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            pil_image = Image.fromarray(rgb_frame).convert("RGB")
            pil_images.append(pil_image)

        frame_count += 1

    cap.release()

    if not pil_images:
        raise RuntimeError("No frames captured from video.")

    pil_images[0].save(pdf_path, save_all=True, append_images=pil_images[1:])
    total_time = time.time() - start_time

    print(f"✅ PDF saved to: {pdf_path}")
    print(f"⏱️ Total time taken: {total_time:.2f} seconds")


In [ ]:
params = {
    "video_path": r"C:\Users\samee\Downloads\Video\xys.mp4",
    "save_path": r"C:\Users\samee\Downloads\Video",  # optional
    "interval": 10,                   # optional
}

video_to_pdf_in_memory(params)